In [5]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 加载数据函数
def load_data(frac=1.0):
    """
    加载用户、样本、行为日志和广告特征数据，并对行为日志进行按比例随机采样。
    
    参数：
    - frac: float, 采样比例，默认为 1.0（加载全部数据）
    
    返回：
    - user: 用户数据
    - sample: 样本数据
    - user_behavior: 用户行为日志数据
    - ad_feature: 广告特征数据
    """
    root_path = "G:/DataSets/Ali_Display_Ad_Click/original_data"
    
    # 加载其他数据
    sample = pd.read_csv(f'{root_path}/raw_sample.csv')
    print("raw_sample读取完毕")
    ad_feature = pd.read_csv(f'{root_path}/ad_feature.csv')
    print("ad_feature读取完毕")
    user = pd.read_csv(f"{root_path}/user_profile.csv")
    print("user_profile读取完毕")
    
    # 加载行为日志数据时进行采样
    print(f"行为日志按 {frac*100:.2f}% 比例采样加载...")
    user_behavior = pd.read_csv(
        f'{root_path}/behavior_log.csv', 
        skiprows=lambda i: i > 0 and np.random.rand() > frac,  # 按随机数过滤
    )
    print("behavior_log加载完毕")
    
    # 填充缺失值
    ad_feature['brand'] = ad_feature['brand'].fillna(-1)
    user['pvalue_level'] = user['pvalue_level'].fillna(-1)
    
    # 修正列名并处理缺失值
    user = user.rename({'new_user_class_level ': 'new_user_class_level'}, axis=1)
    user['new_user_class_level'] = user['new_user_class_level'].fillna(-1)
    
    return user, sample, user_behavior, ad_feature

# 设置采样比例
sampling_fraction = 0.0001

# 加载数据
user, sample, user_behavior, ad_feature = load_data(frac=sampling_fraction)

# 查看加载的数据摘要
print("用户数据：", user.shape)
print("样本数据：", sample.shape)
print("采样后的用户行为数据：", user_behavior.shape)
print("广告特征数据：", ad_feature.shape)

# 显示部分数据
user.head()


raw_sample读取完毕
ad_feature读取完毕
user_profile读取完毕
行为日志按 0.10% 比例采样加载...


ParserError: Error tokenizing data. C error: no error message set

In [ ]:
# 数据预处理函数
def data_preprocess(user_behavior, ad_feature):
    """
    对类别特征进行编码并清洗行为数据中的无效时间戳。
    """
    # 初始化类别编码器
    cate_lbe = LabelEncoder()
    cate_ids = np.concatenate((ad_feature['cate_id'].unique(), user_behavior['cate'].unique()))
    cate_lbe.fit(cate_ids)
    
    # 对类别特征进行编码
    ad_feature['cate_id'] = cate_lbe.transform(ad_feature['cate_id']) + 1
    user_behavior['cate'] = cate_lbe.transform(user_behavior['cate']) + 1
    
    # 品牌特征编码
    brand_lbe = LabelEncoder()
    brand_ids = np.concatenate((ad_feature['brand'].unique(), user_behavior['brand'].unique()))
    brand_lbe.fit(brand_ids)
    ad_feature['brand'] = brand_lbe.transform(ad_feature['brand']) + 1
    user_behavior['brand'] = brand_lbe.transform(user_behavior['brand']) + 1
    
    # 移除时间戳为无效值的行为记录
    user_behavior = user_behavior[user_behavior['time_stamp'] > 0]
    
    return user_behavior, ad_feature

# 数据预处理
user_behavior, ad_feature = data_preprocess(user_behavior, ad_feature)

# 查看预处理后的结果
print("用户行为数据：", user_behavior.shape)
print("广告特征数据：", ad_feature.shape)
user_behavior.head()


In [ ]:
# 抽样函数
def data_sample(frac, user, sample, user_behavior, ad_feature):
    """
    按指定比例对用户进行抽样，并筛选与抽样用户相关的行为和样本数据。
    """
    # 抽样用户
    sel_user = user.sample(frac=frac, random_state=1024).reset_index(drop=True)
    sel_user_ids = sel_user.userid.unique()
    
    # 筛选相关数据
    sel_sample = sample[sample['user'].isin(sel_user_ids)].reset_index(drop=True)
    sel_user_behavior = user_behavior[user_behavior['userid'].isin(sel_user_ids)].reset_index(drop=True)
    sel_ad_feature = ad_feature.copy()  # 广告特征直接复制即可
    
    return sel_user, sel_sample, sel_user_behavior, sel_ad_feature

# 设置抽样比例
frac = 0.001

# 进行数据抽样
sel_user, sel_sample, sel_user_behavior, sel_ad_feature = data_sample(
    frac, user, sample, user_behavior, ad_feature)

# 查看抽样后的结果
print("抽样后的用户数据：", sel_user.shape)
print("抽样后的样本数据：", sel_sample.shape)
print("抽样后的用户行为数据：", sel_user_behavior.shape)


In [ ]:
# 数据保存函数
def save_data(sel_user, sel_sample, sel_user_behavior, sel_ad_feature):
    """
    将抽样和预处理后的数据保存为二进制文件格式。
    """
    exit(1)
    os.makedirs('data1/final_data', exist_ok=True)  # 确保输出目录存在
    sel_user.to_pickle('data1/final_data/user_data.pkl')
    sel_sample.to_pickle('data1/final_data/sample_data.pkl')
    sel_user_behavior.to_pickle('data1/final_data/user_behavior_data.pkl')
    sel_ad_feature.to_pickle('data1/final_data/ad_data.pkl')

# 保存数据
save_data(sel_user, sel_sample, sel_user_behavior, sel_ad_feature)

print("数据已保存到 data1/final_data 文件夹中。")
